In [1]:
file_name="02+03_341dataset_clean" # ชื่อไฟล์คลังข้อมูล

In [2]:
import codecs
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
from nltk.tokenize import RegexpTokenizer
import glob
import nltk
import re
import warnings
warnings.filterwarnings('ignore')

In [3]:
#จัดการประโยคซ้ำ
data_not=[]
def Unique(p):
 text=re.sub("<[^>]*>","",p)
 text=re.sub("\[(.*?)\]","",text)
 text=re.sub("\[\/(.*?)\]","",text)
 if text not in data_not:
  data_not.append(text)
  return True
 else:
  return False
# เตรียมตัวตัด tag ด้วย re
pattern = r'\[(.*?)\](.*?)\[\/(.*?)\]'
tokenizer = RegexpTokenizer(pattern) # ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัด [TIME]8.00[/TIME] ให้เป็น ('TIME','ไง','TIME')
# จัดการกับ tag ที่ไม่ได้ tag
def toolner_to_tag(text):
 text=text.strip()
 text=re.sub("<[^>]*>","",text)
 text=re.sub("(\[\/(.*?)\])","\\1***",text)#.replace('(\[(.*?)\])','***\\1')#  ตัดการกับพวกไม่มี tag word
 text=re.sub("(\[\w+\])","***\\1",text)
 text2=[]
 for i in text.split('***'):
  if "[" in i:
   text2.append(i)
  else:
   text2.append("[word]"+i+"[/word]")
 text="".join(text2)#re.sub("[word][/word]","","".join(text2))
 return text.replace("[word][/word]","")
# แปลง text ให้เป็น conll2002
def text2conll2002(text,pos=True):
    """
    ใช้แปลงข้อความให้กลายเป็น conll2002
    """
    text=toolner_to_tag(text) # นำไปใส่ tag [word]
    text=text.replace("''",'"')
    text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
    tag=tokenizer.tokenize(text) # แยก tag ออกมาจากข้อความ
    j=0
    conll2002="" # ประกาศตัวแปรเก็บ conll2002
    for tagopen,text,tagclose in tag: # ลูปใน tag โดยเป็น (tagopen,text,tagclose)
        word_cut=word_tokenize(text,engine="newmm") # ใช้ตัวตัดคำ newmm ของ PyThaiNLP
        i=0
        txt5=""
        while i<len(word_cut): #ลูปตามจำนวน token ที่ตัดในtag
            if word_cut[i]=="''" or word_cut[i]=='"':pass
            elif i==0 and tagopen!='word': # ไม่เป็น tag [word] และเป็น i หรือตัวเริ่มต้น tag
                txt5+=word_cut[i]
                txt5+='\t'+'B-'+tagopen
            elif tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'I-'+tagopen
            else: # เป็น [word]
                txt5+=word_cut[i]
                txt5+='\t'+'O'
            txt5+='\n'
            #j+=1
            i+=1
        conll2002+=txt5
    if pos==False:
        return conll2002
    return postag(conll2002) # เพิ่ม postag ใส่
# ใช้สำหรับกำกับ pos tag เพื่อใช้กับ NER
# print(text2conll2002(t,pos=False))
def postag(text):
    listtxt=[i for i in text.split('\n') if i!='']
    list_word=[]
    for data in listtxt:
        list_word.append(data.split('\t')[0])
    #print(text)
    list_word=pos_tag(list_word,engine="perceptron")
    text=""
    i=0
    for data in listtxt:
        text+=data.split('\t')[0]+'\t'+list_word[i][1]+'\t'+data.split('\t')[1]+'\n'
        i+=1
    return text
# อ่านข้อมูลจากไฟล์
def get_data(fileopen):
	"""
    สำหรับใช้อ่านทั้งหมดทั้งในไฟล์ทีละรรทัดออกมาเป็น list
    """
	with codecs.open(fileopen, 'r',encoding='utf-8-sig') as f:
		lines = f.read().splitlines()
	return [a for a in lines if Unique(a)] # เอาไม่ซ้ำกัน

def alldata(lists):
    text=""
    for data in lists:
        text+=text2conll2002(data)
        text+='\n'
    return text

def alldata_list(lists):
    data_all=[]
    for data in lists:
        data_num=[]
        try:
            txt=text2conll2002(data,pos=True).split('\n') # นำไปแปลงเป็น conll2002
            for d in txt:
                tt=d.split('\t')
                if d!="":
                    if len(tt)==3:
                        data_num.append((tt[0],tt[1],tt[2]))
                    else:
                        data_num.append((tt[0],tt[1]))
            #print(data_num)
            data_all.append(data_num)
        except:
            print(data)
    #print(data_all)
    return data_all

def alldata_list_str(lists):
	string=""
	for data in lists:
		string1=""
		for j in data:
			string1+=j[0]+"	"+j[1]+"	"+j[2]+"\n"
		string1+="\n"
		string+=string1
	return string

def get_data_tag(listd):
	list_all=[]
	c=[]
	for i in listd:
		if i !='':
			c.append((i.split("\t")[0],i.split("\t")[1],i.split("\t")[2]))
		else:
			list_all.append(c)
			c=[]
	return list_all
def getall(lista):
    ll=[]
    for i in lista:
        o=True
        for j in ll:
            if re.sub("\[(.*?)\]","",i)==re.sub("\[(.*?)\]","",j):
                o=False
                break
        if o==True:
            ll.append(i)
    return ll

In [4]:
data1=getall(get_data(file_name+".txt")) # นำคลังเข้าไป แยกออกเป็น list ละบรรทัด
datatofile=alldata_list(data1) # นำไปผ่านขั้นตอน 1 2 3 4
tt=[]

In [5]:
datatofile[0][:10]

[('มา', 'XVAE', 'O'),
 ('แย้', 'VACT', 'O'),
 ('วววววววววววววววววววววว', 'NCMN', 'O'),
 (' ', 'PUNC', 'O'),
 ('กับ', 'RPRE', 'O'),
 ('รีวิว', 'NCMN', 'O'),
 ('ง่ายๆ', 'VATT', 'O'),
 (' ', 'PUNC', 'O'),
 ('เห็นภาพ', 'NCMN', 'O'),
 ('สดๆ', 'VSTA', 'O')]

In [6]:
len(datatofile)

265

In [7]:
from sklearn_crfsuite import scorers,metrics
from sklearn.metrics import make_scorer, confusion_matrix, classification_report, accuracy_score, make_scorer
from sklearn.model_selection import cross_validate,train_test_split, KFold, StratifiedKFold, cross_val_predict, cross_val_score
import sklearn_crfsuite

In [8]:
def doc2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    # Features from current word
    features={
        'word.word': word,
        'word.isspace':word.isspace(),
        'postag':postag,
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        prevword = doc[i-1][0]
        postag1 = doc[i-1][1]
        features['word.prevword'] = prevword
        features['word.previsspace']=prevword.isspace()
        features['word.prepostag'] = postag1
        features['word.prevwordisdigit'] = prevword.isdigit()
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1][0]
        postag1 = doc[i+1][1]
        features['word.nextword'] = nextword
        features['word.nextisspace']=nextword.isspace()
        features['word.nextpostag'] = postag1
        features['word.nextwordisdigit'] = nextword.isdigit()
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features

def extract_features(doc):
    return [doc2features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [tag for (token,postag,tag) in doc]

# Create features

In [9]:
X_data = [extract_features(doc) for doc in datatofile] 
y_data = [get_labels(doc) for doc in datatofile]
# print(X_data[0][:5])
# print(y_data[0][:20])

In [10]:
X_data[0] #token features ของกระทู้ที่ 1

[{'word.word': 'มา',
  'word.isspace': False,
  'postag': 'XVAE',
  'word.isdigit()': False,
  'BOS': True,
  'word.nextword': 'แย้',
  'word.nextisspace': False,
  'word.nextpostag': 'VACT',
  'word.nextwordisdigit': False},
 {'word.word': 'แย้',
  'word.isspace': False,
  'postag': 'VACT',
  'word.isdigit()': False,
  'word.prevword': 'มา',
  'word.previsspace': False,
  'word.prepostag': 'XVAE',
  'word.prevwordisdigit': False,
  'word.nextword': 'วววววววววววววววววววววว',
  'word.nextisspace': False,
  'word.nextpostag': 'NCMN',
  'word.nextwordisdigit': False},
 {'word.word': 'วววววววววววววววววววววว',
  'word.isspace': False,
  'postag': 'NCMN',
  'word.isdigit()': False,
  'word.prevword': 'แย้',
  'word.previsspace': False,
  'word.prepostag': 'VACT',
  'word.prevwordisdigit': False,
  'word.nextword': ' ',
  'word.nextisspace': True,
  'word.nextpostag': 'PUNC',
  'word.nextwordisdigit': False},
 {'word.word': ' ',
  'word.isspace': True,
  'postag': 'PUNC',
  'word.isdigit()': 

In [11]:
print(str(len(X_data))+' , '+str(len(y_data)))

265 , 265


In [12]:
y_data[0][:2]

['O', 'O']

# KFold = 10-Fold

In [13]:
X, X_test, y, y_test = train_test_split(X_data, y_data, test_size=0.1) # แบ่ง 0.1 หรือ 10%
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=500,
    all_possible_transitions=True,
#     model_filename=file_name+"-pos.model0" # ตั้งชื่อโมเดล
)
crf.fit(X, y); # train

In [14]:
print(X[0])
print(y[0])

[{'word.word': 'ช่วงนี้', 'word.isspace': False, 'postag': 'NCMN', 'word.isdigit()': False, 'BOS': True, 'word.nextword': ' ', 'word.nextisspace': True, 'word.nextpostag': 'PUNC', 'word.nextwordisdigit': False}, {'word.word': ' ', 'word.isspace': True, 'postag': 'PUNC', 'word.isdigit()': False, 'word.prevword': 'ช่วงนี้', 'word.previsspace': False, 'word.prepostag': 'NCMN', 'word.prevwordisdigit': False, 'word.nextword': 'เด็กน้อย', 'word.nextisspace': False, 'word.nextpostag': 'NCMN', 'word.nextwordisdigit': False}, {'word.word': 'เด็กน้อย', 'word.isspace': False, 'postag': 'NCMN', 'word.isdigit()': False, 'word.prevword': ' ', 'word.previsspace': True, 'word.prepostag': 'PUNC', 'word.prevwordisdigit': False, 'word.nextword': 'เริ่ม', 'word.nextisspace': False, 'word.nextpostag': 'VSTA', 'word.nextwordisdigit': False}, {'word.word': 'เริ่ม', 'word.isspace': False, 'postag': 'VSTA', 'word.isdigit()': False, 'word.prevword': 'เด็กน้อย', 'word.previsspace': False, 'word.prepostag': 'NCMN

In [15]:
labels = list(crf.classes_)
labels.remove('O')
# e=metrics.flat_f1_score(y_test, y_pred,
#                       average='weighted', labels=labels)
# print(e) # โชว์ประสิทธิภาพ

In [16]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

In [17]:
sorted_labels

['B-HOTEL',
 'I-HOTEL',
 'B-LOCATION',
 'I-LOCATION',
 'B-RESTAURANT',
 'I-RESTAURANT']

### No 10-Fold Classification Report

In [18]:
y_pred = crf.predict(X_test)

In [19]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

     B-HOTEL      0.727     0.250     0.372        32
     I-HOTEL      0.701     0.448     0.547       105
  B-LOCATION      0.830     0.556     0.666       595
  I-LOCATION      0.846     0.515     0.640      1240
B-RESTAURANT      0.750     0.094     0.167        32
I-RESTAURANT      0.652     0.195     0.300        77

   micro avg      0.828     0.501     0.624      2081
   macro avg      0.751     0.343     0.449      2081
weighted avg      0.824     0.501     0.619      2081



### 10-Fold Classification Report

In [20]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=10)

In [21]:
report = metrics.flat_classification_report(y_pred=pred, y_true=y, labels=sorted_labels, digits=3)
print(report)

              precision    recall  f1-score   support

     B-HOTEL      0.634     0.141     0.231       319
     I-HOTEL      0.669     0.324     0.436      1514
  B-LOCATION      0.809     0.584     0.678      4993
  I-LOCATION      0.803     0.626     0.704      8876
B-RESTAURANT      0.684     0.208     0.319       250
I-RESTAURANT      0.536     0.212     0.304      1067

   micro avg      0.785     0.546     0.644     17019
   macro avg      0.689     0.349     0.445     17019
weighted avg      0.771     0.546     0.633     17019



## Merge Label

In [22]:
print(len(y), len(pred))

238 238


In [23]:
pred[:2]

array([list(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [24]:
y[:2]

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOCATION',
  'I-LOCATION',
  'I-LOCATION',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOCATION',
  'I-LOCATION',
  'I-LOCATION',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
 

In [25]:
pred_loc_index = []
pred_res_index = []
pred_hot_index = []
for i in range(len(pred)): #วน ทีละกระทู้
    for j in range(len(pred[i])): #วน ภายใน 1 กระทู้
        if(pred[i][j]=='B-LOCATION'):
            pred_loc_index.append(('B-LOCATION',i,j))
        elif(pred[i][j]=='I-LOCATION'):
            pred_loc_index.append(('I-LOCATION',i,j))
        elif(pred[i][j]=='B-RESTAURANT'):
            pred_res_index.append(('B-RESTAURANT',i,j))
        elif(pred[i][j]=='I-RESTAURANT'):
            pred_res_index.append(('I-RESTAURANT',i,j))
        elif(pred[i][j]=='B-HOTEL'):
            pred_hot_index.append(('B-HOTEL',i,j))
        elif(pred[i][j]=='I-HOTEL'):
            pred_hot_index.append(('I-HOTEL',i,j))

In [40]:
print(len(pred_loc_index))
print(pred_loc_index)

10523
[('B-LOCATION', 0, 39), ('I-LOCATION', 0, 40), ('I-LOCATION', 0, 41), ('B-LOCATION', 0, 98), ('I-LOCATION', 0, 99), ('I-LOCATION', 0, 100), ('B-LOCATION', 0, 532), ('B-LOCATION', 0, 578), ('B-LOCATION', 1, 13), ('B-LOCATION', 1, 22), ('B-LOCATION', 2, 16), ('I-LOCATION', 2, 17), ('I-LOCATION', 2, 18), ('I-LOCATION', 2, 19), ('I-LOCATION', 2, 20), ('B-LOCATION', 2, 111), ('B-LOCATION', 2, 117), ('B-LOCATION', 3, 24), ('I-LOCATION', 3, 25), ('I-LOCATION', 3, 26), ('B-LOCATION', 3, 44), ('I-LOCATION', 3, 45), ('I-LOCATION', 3, 46), ('B-LOCATION', 3, 118), ('I-LOCATION', 3, 119), ('B-LOCATION', 3, 214), ('I-LOCATION', 3, 215), ('B-LOCATION', 3, 227), ('I-LOCATION', 3, 228), ('I-LOCATION', 3, 229), ('I-LOCATION', 3, 230), ('I-LOCATION', 3, 231), ('I-LOCATION', 3, 232), ('B-LOCATION', 3, 249), ('I-LOCATION', 3, 250), ('I-LOCATION', 3, 251), ('B-LOCATION', 3, 322), ('I-LOCATION', 3, 323), ('I-LOCATION', 3, 324), ('B-LOCATION', 3, 328), ('I-LOCATION', 3, 329), ('I-LOCATION', 3, 330), ('I

In [41]:
print(len(pred_res_index))
print(pred_res_index)

498
[('B-RESTAURANT', 5, 2777), ('I-RESTAURANT', 5, 2778), ('I-RESTAURANT', 5, 2779), ('B-RESTAURANT', 12, 1078), ('I-RESTAURANT', 12, 1079), ('I-RESTAURANT', 12, 1080), ('B-RESTAURANT', 19, 560), ('I-RESTAURANT', 19, 561), ('I-RESTAURANT', 19, 562), ('B-RESTAURANT', 34, 214), ('I-RESTAURANT', 34, 215), ('I-RESTAURANT', 34, 216), ('I-RESTAURANT', 34, 217), ('I-RESTAURANT', 34, 218), ('I-RESTAURANT', 34, 219), ('I-RESTAURANT', 34, 220), ('I-RESTAURANT', 34, 221), ('I-RESTAURANT', 34, 222), ('I-RESTAURANT', 34, 223), ('B-RESTAURANT', 34, 287), ('I-RESTAURANT', 34, 288), ('I-RESTAURANT', 34, 289), ('I-RESTAURANT', 34, 290), ('I-RESTAURANT', 34, 291), ('I-RESTAURANT', 34, 292), ('I-RESTAURANT', 34, 293), ('I-RESTAURANT', 34, 294), ('I-RESTAURANT', 34, 295), ('I-RESTAURANT', 34, 296), ('I-RESTAURANT', 34, 297), ('I-RESTAURANT', 34, 298), ('I-RESTAURANT', 34, 299), ('I-RESTAURANT', 34, 300), ('I-RESTAURANT', 34, 301), ('I-RESTAURANT', 34, 302), ('I-RESTAURANT', 34, 303), ('I-RESTAURANT', 34,

In [42]:
print(len(pred_hot_index))
print(pred_hot_index)

803
[('B-HOTEL', 5, 141), ('I-HOTEL', 5, 142), ('I-HOTEL', 5, 143), ('B-HOTEL', 5, 706), ('I-HOTEL', 5, 707), ('I-HOTEL', 5, 708), ('I-HOTEL', 5, 709), ('I-HOTEL', 5, 710), ('B-HOTEL', 5, 2303), ('I-HOTEL', 5, 2304), ('I-HOTEL', 5, 2305), ('B-HOTEL', 10, 524), ('I-HOTEL', 10, 525), ('I-HOTEL', 10, 526), ('I-HOTEL', 10, 527), ('I-HOTEL', 10, 528), ('I-HOTEL', 10, 529), ('I-HOTEL', 10, 530), ('I-HOTEL', 10, 531), ('I-HOTEL', 10, 532), ('I-HOTEL', 10, 533), ('I-HOTEL', 10, 534), ('I-HOTEL', 10, 535), ('I-HOTEL', 10, 536), ('B-HOTEL', 10, 647), ('I-HOTEL', 10, 648), ('I-HOTEL', 10, 649), ('B-HOTEL', 11, 65), ('I-HOTEL', 11, 66), ('I-HOTEL', 11, 67), ('I-HOTEL', 11, 68), ('I-HOTEL', 11, 69), ('I-HOTEL', 11, 70), ('I-HOTEL', 11, 71), ('I-HOTEL', 11, 72), ('I-HOTEL', 11, 73), ('I-HOTEL', 11, 74), ('I-HOTEL', 11, 75), ('I-HOTEL', 11, 76), ('I-HOTEL', 11, 77), ('I-HOTEL', 11, 78), ('B-HOTEL', 19, 79), ('I-HOTEL', 19, 80), ('I-HOTEL', 19, 81), ('I-HOTEL', 19, 82), ('I-HOTEL', 19, 83), ('I-HOTEL'

In [54]:
print(X[0][39:42])
print(X[12][235:237])

[{'word.word': 'สวนสัตว์', 'word.isspace': False, 'postag': 'NCMN', 'word.isdigit()': False, 'word.prevword': 'ใน', 'word.previsspace': False, 'word.prepostag': 'RPRE', 'word.prevwordisdigit': False, 'word.nextword': 'เขา', 'word.nextisspace': False, 'word.nextpostag': 'PPRS', 'word.nextwordisdigit': False}, {'word.word': 'เขา', 'word.isspace': False, 'postag': 'PPRS', 'word.isdigit()': False, 'word.prevword': 'สวนสัตว์', 'word.previsspace': False, 'word.prepostag': 'NCMN', 'word.prevwordisdigit': False, 'word.nextword': 'เขียว', 'word.nextisspace': False, 'word.nextpostag': 'VATT', 'word.nextwordisdigit': False}, {'word.word': 'เขียว', 'word.isspace': False, 'postag': 'VATT', 'word.isdigit()': False, 'word.prevword': 'เขา', 'word.previsspace': False, 'word.prepostag': 'PPRS', 'word.prevwordisdigit': False, 'word.nextword': ' ', 'word.nextisspace': True, 'word.nextpostag': 'PUNC', 'word.nextwordisdigit': False}]
[{'word.word': 'เมือง', 'word.isspace': False, 'postag': 'VACT', 'word.isd

In [31]:
true_loc_index = []
true_res_index = []
true_hot_index = []
for i in range(len(pred)): #วน ทีละกระทู้
    for j in range(len(pred[i])): #วน ภายใน 1 กระทู้
        if(y[i][j]=='B-LOCATION'):
            true_loc_index.append(('B-LOCATION',i,j))
        elif(y[i][j]=='I-LOCATION'):
            true_loc_index.append(('I-LOCATION',i,j))
        elif(y[i][j]=='B-RESTAURANT'):
            true_res_index.append(('B-RESTAURANT',i,j))
        elif(y[i][j]=='I-RESTAURANT'):
            true_res_index.append(('I-RESTAURANT',i,j))
        elif(y[i][j]=='B-HOTEL'):
            true_hot_index.append(('B-HOTEL',i,j))
        elif(y[i][j]=='I-HOTEL'):
            true_hot_index.append(('I-HOTEL',i,j))

In [32]:
true_loc_index[0][0]

'B-LOCATION'

In [33]:
len(pred_loc_index)

10523

### นับ predict ที่ตรงกับ true

In [34]:
count_loc = 0
state_loc = 0
for pl in pred_loc_index:  #pl = ('B-LOCATION', 3, 14) => กระทู้ 3 คำที่ 14 เป็น B_LOCATION
    if(pl[0]=='B-LOCATION' and (pl in true_loc_index)):
        count_loc += 1 #เจอ B แล้วบวก 1 เลย => เจอ 1 คำ
        state_loc = 1 #ไว้บอกคำต่อไปว่าก่อนหน้าเป็น B-LOCATION
    elif(pl[0]=='I-LOCATION' and (pl not in true_loc_index) and state_loc==1):
        count_loc -= 1 #ถ้ามี I ไม่ครบในคำนั้น => ลบคำนั้นทิ้งไป 
        state_loc = 0
    
count_res = 0
state_res = 0
for pr in pred_res_index:
    if(pr[0]=='B-RESTAURANT' and (pr in true_res_index)):
        count_res += 1 #เจอ B แล้วบวก 1 เลย => เจอ 1 คำ
        state_res = 1 #ไว้บอกคำต่อไปว่าก่อนหน้าเป็น B-LOCATION
    elif(pr[0]=='I-RESTAURANT' and (pr not in true_res_index) and state_res==1):
        count_res -= 1 #ถ้ามี I ไม่ครบในคำนั้น => ลบคำนั้นทิ้งไป 
        state_res = 0
        
count_hot = 0
state_hot = 0
for ph in pred_hot_index:
    if(ph[0]=='B-HOTEL' and (ph in true_hot_index)):
        count_hot += 1 #เจอ B แล้วบวก 1 เลย => เจอ 1 คำ
        state_hot = 1 #ไว้บอกคำต่อไปว่าก่อนหน้าเป็น B-LOCATION
    elif(ph[0]=='I-HOTEL' and (ph not in true_hot_index) and state_hot==1):
        count_hot -= 1 #ถ้ามี I ไม่ครบในคำนั้น => ลบคำนั้นทิ้งไป 
        state_hot = 0

In [35]:
print(count_loc, count_res, count_hot)

2476 33 24


### จำนวนคำ True Location

In [36]:
count_true_loc = 0
for i in true_loc_index:
    if(i[0]=='B-LOCATION'):
        count_true_loc += 1

count_true_res = 0
for j in true_res_index:
    if(j[0]=='B-RESTAURANT'):
        count_true_res += 1
        
count_true_hot = 0
for k in true_hot_index:
    if(k[0]=='B-HOTEL'):
        count_true_hot += 1

In [37]:
print(count_true_loc, count_true_res, count_true_hot)

4993 250 319


## Score

In [38]:
score_loc = count_loc / count_true_loc
score_res = count_res / count_true_res
score_hot = count_hot / count_true_hot

In [39]:
print(score_loc, score_res, score_hot)

0.49589425195273384 0.132 0.07523510971786834


# Test

In [51]:
mw = X[0][39]['word.word']+X[0][40]['word.word']+X[0][41]['word.word']
mw

'สวนสัตว์เขาเขียว'

In [64]:
mw = X[24][625]['word.word']+X[24][626]['word.word']
mw

'สิงห์ปาร์ค'

In [53]:
mw = X[12][235]['word.word']+X[12][236]['word.word']
mw

'เมืองเชียงใหม่'

In [57]:
mw = X[5][2777]['word.word']+X[5][2778]['word.word']+X[5][2779]['word.word']
mw

'ร้านพรเทพ'

In [58]:
mw = X[49][119]['word.word']+X[5][120]['word.word']+X[5][121]['word.word'] ########
mw

'ร้าน ครั้งนี้'

In [62]:
mw = X[5][141]['word.word']+X[5][142]['word.word']+X[5][143]['word.word']
mw

'The River'

In [63]:
mw = X[58][1767]['word.word']+X[58][1768]['word.word']+X[58][1769]['word.word']+X[58][1770]['word.word']+X[58][1771]['word.word']+X[58][1772]['word.word']+X[58][1773]['word.word']
mw

'ทีลอซู ริเวอร์ไซด์ รีสอร์ท'